In [1]:
import json
import math
import requests
import pandas as pd
from datetime import datetime, timedelta
from Creds import *
from tqdm import tqdm

In [2]:
mainZipCode = 80309


In [3]:
# aerisWeatherURL = f'https://api.aerisapi.com/conditions/80309?format=json&from={fromDateStr}&to={toDateStr}&plimit=24&filter=1hr&fields=periods.tempF,periods.dateTimeISO,periods.dewpointF,periods.windSpeedMPH,periods.windDir,periods.weather&client_id={clientID}&client_secret={clientSecret}'



In [4]:
def CalculateCompassAngle(fromLat: float, fromLong: float, toLat: float, toLong: float) -> int:
    fromLat = math.radians(fromLat)
    toLat = math.radians(toLat)
    longDiff = math.radians(toLong - fromLong)

    x = math.sin(longDiff) * math.cos(toLat)
    y = math.cos(fromLat) * math.sin(toLat) - (math.sin(fromLat) * math.cos(toLat) * math.cos(longDiff))
    actualAngle = math.atan2(x, y)

    actualAngle = math.degrees(actualAngle)
    compassAngle = (actualAngle + 360) % 360

    return int(compassAngle)

In [5]:
def GetCompassDirection(angle: int) -> str:
    directions = ["North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest"]
    return directions[int((angle + 22.5) % 360 / 45)]


In [6]:
def GetNearbyZipCodes(presentZipcode: int,  geonamesUsername: str, radius: int = 30, maxRows: int = 100, country: str = 'US', minimumDistance: int = 10  ) -> dict:
    geonamesURL = f'http://api.geonames.org/findNearbyPostalCodesJSON?postalcode={presentZipcode}&maxRows={maxRows}&country={country}&radius={radius}&username={geonamesUsername}'
    allZipcodes = []
    responseJson = {}
    nearestZipCodes = {'North': None, 'East': None, 'South': None, 'West': None}
    nearestDistances = {'North': minimumDistance, 'East': minimumDistance, 'South': minimumDistance, 'West': minimumDistance}
    
    response = requests.get(geonamesURL)
    if(response.status_code == 200):
        responseJson = response.json()
    else:
        return {"Error" :str(response.status_code)}
    
    locationsDF = pd.DataFrame(responseJson['postalCodes'])
    locationsDF = locationsDF[['lat','lng', 'distance', 'postalCode' ]]
    locationsDF = locationsDF.astype({'lat': 'float', 'lng': 'float', 'distance': 'float', 'postalCode': 'float'}, errors='ignore')
    fromLocation = locationsDF[locationsDF['postalCode'] == presentZipcode].values[0]
    nearByLocations = locationsDF[locationsDF['postalCode'] != presentZipcode ].values
    
    for location in nearByLocations:
        compassAngle = CalculateCompassAngle(fromLocation[0], fromLocation[1], location[0], location[1])
        direction = GetCompassDirection(compassAngle)
        if(direction in  ["North", "East", "South", "West"]):
            allZipcodes.append({"zipCode": location[3],'direction': direction, 'distance':location[2] })
    
    for zipcode in allZipcodes:
        if(zipcode['distance'] > nearestDistances[zipcode['direction']]):
            nearestDistances[zipcode['direction']] = zipcode['distance']
            nearestZipCodes[zipcode['direction']] = int(zipcode['zipCode'])
    print(nearestDistances)
    nearestZipCodes['currentLocation'] = presentZipcode
    return nearestZipCodes
        
nearestZipCodes = GetNearbyZipCodes(mainZipCode, GEONAMESUSERNAME  )
print(nearestZipCodes)

     

{'North': 24.94578, 'East': 28.22693, 'South': 29.15415, 'West': 21.20428}
{'North': 80540, 'East': 80614, 'South': 80402, 'West': 80481, 'currentLocation': 80309}


In [7]:
#  {
#     "dateTimeISO": "2023-11-14T15:59:00-07:00",
#     "tempF": 68.81,
#     "dewpointF": 8.17,
#     "weather": "Sunny",
#     "windSpeedMPH": 3.66,
#     "windDir": "SSW"
# }
# {'North': 80304, 'East': 80027, 'South': 80305, 'West': 80308, 'currentLocation': 80309}



In [8]:
def GetWeatherDatafromZipCode(zipcode: int, fromDateStr: str, toDateStr: str, clientID: str, clientSecret: str ):
    aerisWeatherURL = f'https://api.aerisapi.com/conditions/{zipcode}?format=json&from={fromDateStr}&to={toDateStr}&plimit=24&filter=1hr&fields=periods.tempF,periods.dateTimeISO,periods.dewpointF,periods.windSpeedMPH,periods.windDir,periods.weather&client_id={clientID}&client_secret={clientSecret}'
    response = requests.get(aerisWeatherURL)
    if(response.status_code == 200):
        weatherReport = response.json()['response'][0]['periods']
        # weatherReport['zipcode'] = zipcode
        return weatherReport
    else:
        return [{"Error" :str(response.status_code)}]

# weatherReport = GetWeatherDatafromZipCode(80309, "2017/08/01", "2017/08/02", "7XKKvv15LCntut54UVLdR","Z3GLu5I6YAJ7wNuo1E4m02PSZ0efLN643Fhr66yx")
# print(weatherReport)


In [9]:
datasetStartDate = datetime(2017, 12, 16)
datasetEndDate = datetime(2023, 11, 11)
APISECRETSindex = 0
fromDate = datasetStartDate
dates = []
while(fromDate < datasetEndDate ):
    dates.append(fromDate)
    fromDate = fromDate+timedelta(days=1)

    
    
for fromDate in tqdm(dates):
    weatherDetails = {}
    toDate = fromDate+timedelta(days=1)
    fromDateStr = fromDate.strftime('%Y/%m/%d')
    toDateStr = toDate.strftime('%Y/%m/%d')
    # print("Collecting data on",fromDateStr )
    for direction, zipcode in nearestZipCodes.items():
        # print("Collecting data at", direction, "direction")
        weatherReport = GetWeatherDatafromZipCode(zipcode, fromDateStr, toDateStr, APISECRETS[APISECRETSindex]['clientID'],APISECRETS[APISECRETSindex]['clientSecret'])
        if(weatherReport[0].get("Error") is not None):
            print(weatherReport[0]["Error"])
            APISECRETSindex += 1
            weatherReport = GetWeatherDatafromZipCode(zipcode, fromDateStr, toDateStr, APISECRETS[APISECRETSindex]['clientID'],APISECRETS[APISECRETSindex]['clientSecret']) 
               
        for weather in weatherReport:
            recoredDate = weather['dateTimeISO']
            recoredDate = datetime.fromisoformat(recoredDate)
            recoredDate = recoredDate.strftime("%Y-%m-%d %H:%M:%S")
            
            if(weatherDetails.get(recoredDate) is None):
                weatherDetails[recoredDate] = {}
            for key, value in weather.items():
                if(key == 'dateTimeISO' ):
                    pass
                else:
                    weatherDetails[recoredDate][f'{key}_{direction}'] = value
    jsonFileName = fromDate.strftime('%Y-%m-%d')
    with open(f'Dataset/Weather Dataset/{jsonFileName}.json', 'w') as json_file:
        json.dump(weatherDetails, json_file, indent=4)
            
    
    



  0%|          | 1/2156 [00:00<17:34,  2.04it/s]

429


 12%|█▏        | 263/2156 [41:06<4:29:53,  8.55s/it]

429


 18%|█▊        | 380/2156 [58:58<4:19:08,  8.75s/it]

...

 83%|████████▎ | 1796/2156 [5:53:23<1:19:43, 13.29s/it]

429


 94%|█████████▎| 2020/2156 [6:45:47<33:02, 14.58s/it]  

429


100%|██████████| 2156/2156 [7:14:16<00:00, 12.09s/it]
